In [ ]:
!pip install openai
!pip install python-dotenv

In [ ]:
# Crie um arquivo .env e adicione a chave de API
with open(".env", "w") as f:
    f.write("API_KEY")

In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI

# Carregue as variáveis de ambiente do arquivo .env
load_dotenv()

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.environ.get("OPENAI_API_KEY"),
)

client = OpenAI()

# Passo 3: Faça uma chamada à API usando a nova interface
stream = client.chat.completions.create(
    model="gpt-4",
    messages=[{"role": "user", "content": "Say this is a test"}],
    stream=True,
)
for chunk in stream:
    print(chunk.choices[0].delta.content or "", end="")

This is a test.

In [ ]:
pip install pydub ffmpeg-python moviepy

In [ ]:
!apt-get install -y ffmpeg

In [ ]:
pip install --upgrade moviepy

In [ ]:
pip install pydub


In [ ]:
from pydub import AudioSegment
import os

# Caminho para o arquivo MP4
input_video = "/content/modelos.mp4"

# Caminho para o arquivo de saída MP3
output_audio = "/content/modelos.mp3"

# Carregar o áudio do vídeo MP4
audio = AudioSegment.from_file(input_video, "mp4")

# Exportar o áudio como MP3
audio.export(output_audio, format="mp3")

print("Conversão concluída com sucesso!")


CouldntDecodeError: Decoding failed. ffmpeg returned error code: 1

Output from ffmpeg/avlib:

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --ena

In [ ]:
def split_audio(file_path, output_folder, max_size=25 * 1024 * 1024):
    audio = AudioSegment.from_file(file_path)
    # Taxa de bits média em bits por segundo (bps)
    bitrate = 128 * 1024  # 128 kbps
    max_duration_ms = (max_size * 8 * 1000) / bitrate  # Converter para milissegundos

    os.makedirs(output_folder, exist_ok=True)

    parts = []
    part_index = 1
    start_ms = 0

    while start_ms < len(audio):
        end_ms = start_ms + max_duration_ms
        if end_ms > len(audio):
            end_ms = len(audio)

        part = audio[start_ms:end_ms]
        part_path = os.path.join(output_folder, f"part_{part_index}.mp3")
        part.export(part_path, format="mp3")

        parts.append(part_path)
        part_index += 1
        start_ms = end_ms

    return parts

# Caminho para o arquivo MP3
input_audio = "/content/Modelos.mp3"

# Pasta para salvar os arquivos divididos
output_folder = "/content/div3"

# Dividir o áudio
parts = split_audio(input_audio, output_folder)

print("Divisão concluída com sucesso!")
print("Partes geradas:", parts)

Divisão concluída com sucesso!
Partes geradas: ['/content/div3/part_1.mp3']


In [ ]:
def transcribe_audio_part(audio_path):
    with open(audio_path, "rb") as audio_file:
        transcription = client.audio.transcriptions.create(
          model="whisper-1",
          file=audio_file
        )
        return transcription.text

def transcribe_and_concatenate(audio_folder, max_parts=10):
    transcriptions = []
    audio_files = sorted([os.path.join(audio_folder, f) for f in os.listdir(audio_folder) if f.endswith('.mp3')])
    for i, part in enumerate(audio_files):
        if i >= max_parts:
            break
        print(f"Transcrevendo parte {i + 1} de {min(len(audio_files), max_parts)}")
        text = transcribe_audio_part(part)
        transcriptions.append(text)
    return " ".join(transcriptions)

# Função para salvar a transcrição em um arquivo .txt
def save_transcription_to_file(transcription, file_path):
    with open(file_path, "w") as file:
        file.write(transcription)

# Pasta contendo as partes de áudio divididas
audio_folder = "/content/div3"

# Limitar a 10 partes para teste
final_transcription = transcribe_and_concatenate(audio_folder, max_parts=5)

# Caminho do arquivo .txt para salvar a transcrição
output_file_path = "/content/modelos.txt"

# Salvar a transcrição no arquivo .txt
save_transcription_to_file(final_transcription, output_file_path)

print("Transcrição final:")
print(final_transcription)

Transcrevendo parte 1 de 1
Transcrição final:
Nós convidamos o Roberto Figueira, Roberto, você já conhece, e só para eu contar qual é o título, qual é o tema, o Roberto Figueira, ele é Superintendente de Engenharia da Plataforma de Dados e Ferramentas Analíticas do Itaú Unibanco. Ele é bacharel, mestre e doutor em Ciências de Computação. Possui mais de 15 anos de experiência no mercado financeiro, também se ele souber como é o nome disso. Passando por projetos nas áreas de dados, de metadata, de plataforma de risco de crédito, de data house, de BI, de HPC e modernização e mainstream. Nos últimos dois anos se testificou a mitigar o data lake para a nuvem e estruturação e implementação do modelo arquitetural diretamente na corporação. O Roberto vai falar hoje, gente, do modelo de IA, que é só a ponta do iceberg. O que está nos bastidores e algumas dicas que eu acho que é o mais importante para organizar os seus dados com sucesso. Então, Roberto, você. Obrigada, gente. Obrigado, Silvana. 